# Plate Reader Exploration Data Generator

This notebook uses the code in the `BiochemToolbox' library to generate data using setup data files. 

Each plate in the dataset has a name in the format "plateXX" where XX is a number between 00 and 99. A setup file is named according to the plate name as `setup/PlateXX_setup.csv`. Plates are arranged so that each well in a column (lane) contains a set amount of a given enzyme and each well in a row contains a set amount of substrate. The stup file will contain data that describes each lane (lane label, enzyme name, and enzyme concentration) and data that describes each row (row label, substrate concentration). This same data makes up the plate plan file used by the student in this exercise. The setup file contains additional data for each enzyme ($k_{cat}$ and $K_M$ values).

In both the plate plan files and the plate setup files the numbers have the following units: $K_M$ values and substrate concentrations are millimolar; enzyme concentrations is nanomolar; and $k_{cat}$ is $s^{-1}$. These units are expected by the code used to generate and analyze the data. Units are also describes in the comment lines in each file.

## Make the Data

The code below will use the `make_data()` function to calculate the abs vs time data for each well in a plate. 

In [1]:
import numpy as np
import pandas as pd

#!wget https://raw.githubusercontent.com/blinkletter/StealThisCode/main/BiochemToolbox.py
#!pip install uncertainties  # to install 'Uncertainties' package in Colab. 

import BiochemToolbox as BT

file_location = "data2/"
data_dir = "data/"
setup_dir = "setup/"
plot_dir = "plots/"

plates = np.arange(0,13)   # gives an array of integers from 0 to 12
print(f"Plates: {plates}")

for a in plates:   
    plate_name = "Plate"+f"{str(a):0>2}"
    setup_file_name = file_location + setup_dir + plate_name + "_setup.csv"
    outfile_rootname = file_location + data_dir + plate_name
    BT.make_data(setup_file_name, outfile_rootname, pH = 7.0)
    print(f"{plate_name} done")


Plates: [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
Plate00 done
Plate01 done
Plate02 done
Plate03 done
Plate04 done
Plate05 done
Plate06 done
Plate07 done
Plate08 done
Plate09 done
Plate10 done
Plate11 done
Plate12 done


## Check Data

The code below will output a contact sheet of plots for each plate.  Stacked abs vs time plots are presented for each column of the plate.


In [2]:
import BiochemToolbox as BT

file_location = "data2/"
data_dir = "data/"
setup_dir = "setup/"
plot_dir = "plots/"

#for a in range(2):
for a in plates:

    plate_name = "Plate"+f"{str(a):0>2}"
    BT.contact_sheet(file_location+data_dir+plate_name, fancy = True)

Plot saved as data2/plots/plot_contact_sheet_00.pdf
Plot saved as data2/plots/plot_contact_sheet_01.pdf
Plot saved as data2/plots/plot_contact_sheet_02.pdf
Plot saved as data2/plots/plot_contact_sheet_03.pdf
Plot saved as data2/plots/plot_contact_sheet_04.pdf
Plot saved as data2/plots/plot_contact_sheet_05.pdf
Plot saved as data2/plots/plot_contact_sheet_06.pdf
Plot saved as data2/plots/plot_contact_sheet_07.pdf
Plot saved as data2/plots/plot_contact_sheet_08.pdf
Plot saved as data2/plots/plot_contact_sheet_09.pdf
Plot saved as data2/plots/plot_contact_sheet_10.pdf
Plot saved as data2/plots/plot_contact_sheet_11.pdf
Plot saved as data2/plots/plot_contact_sheet_12.pdf


## Other Tools

The code blocks below are used for various housekeeping functions. Below we see code to create lists of random enzyme names and kinetic parameters. These were used in making the setup files and plate plans.

In [23]:
### generate random names for mutant enzymes.
### Cut and paste the output as you build the setup files
### for each plate

import random
import string

for x in range(11):
   random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=3))
   print(f"{random_string: >10},")   #print string within block of 10 spaces, right formatted



       6FQ,
       0GS,
       LL5,
       GMR,
       K92,
       SRI,
       NYU,
       KQX,
       H5E,
       RZ0,
       1KV,


In [13]:
### generate random numbers for kcat and KM values.
### Cut and paste the output as you build the setup files
### for each plate

import random

for kcat in random.sample(range(15, 233), 11):
    KM = random.uniform(0.01, 0.80)
    print(f"{kcat:10.0f},      {KM:0.2f}")

       225,      0.36
        41,      0.35
       127,      0.54
       168,      0.15
        64,      0.63
       150,      0.75
        52,      0.29
       146,      0.80
       120,      0.31
        19,      0.47
        51,      0.35


In [13]:
for a in range(13):
    print(f"{str(a):0>2}")

00
01
02
03
04
05
06
07
08
09
10
11
12
